**Imports, Initializations, and Helping Functions:**

In [1]:

import random
import numpy as np
import os

size = 3
Board = np.zeros((size, size), dtype = np.int32)
HUMAN = -1
COMPUTER = 1
DRAW = 0
CLEAR = "clear" if os.name == "posix" else "cls"
INF = float("inf")

def LeftBorder(i, size):
    if i == 0:
        return "┌"
    elif i == size:
        return "└"
    else:
        return "├"


def RightBorder(i, size):
    if i == 0:
        return "┐"
    elif i == size:
        return "┘"
    else:
        return "┤"


def MiddleBorder(i, size):
    if i == 0:
        return "┬"
    elif i == size:
        return "┴"
    else:
        return "┼"

def BordersBuilder(i, size):
    ans = LeftBorder(i, size)
    ans += MiddleBorder(i, size).join(["───" for _ in range(size)])
    ans += RightBorder(i, size)
    return ans
def SingleCol(Board, i):
    print("   │ ", end="")
    for j in range(size):
        value = Board[i, j]
        chArr = [" ", "O", "X"][value]
        print(chArr + " │ ", end="")

def SingleRow(Board):
    print("\n   " + BordersBuilder(0, size))
    for i in range(size):
        print("   │ ", end="")
        for j in range(size):
            value = Board[i, j]
            chArr = [" ", "O", "X"][value]
            print(chArr + " │ ", end="")
        print("\n   " + BordersBuilder(i + 1, size))

def printBoard(Board):
    os.system(CLEAR)
    print("     ", end="")
    for col in range(size):
        print(end="   ")
    SingleRow(Board)
    print()



**Main Algorithm with Alpha Beta Pruning**

In [2]:


def SetMove(Board, row, col, player):
    Board[row][col] = player

def empty_Cells(Board):
    blanks = np.where(Board == 0)
    blanks = np.transpose(blanks)
    np.random.shuffle(blanks)
    return blanks

def win(Board, player):
    result = player * size in Board.sum(axis=1)
    result = result or player * size in Board.sum(axis=0)
    result = result or player * size == np.trace(Board)
    result = result or player * size == np.trace(np.fliplr(Board))
    return result

def GameOver(Board):
    return win(Board, HUMAN) or win(Board, COMPUTER) or DRAW not in Board

def evaluate(Board):
    if win(Board, HUMAN):
        return HUMAN
    if win(Board, COMPUTER):
        return COMPUTER
    return DRAW

def MiniMaxAlphaBeta(Board, computer, alpha, beta, depth):
    if GameOver(Board):
        return evaluate(Board), depth
    if computer:
        func = max
        m = -INF
        char = COMPUTER
    else:
        func = min
        m = INF
        char = HUMAN
    for empty_cell in empty_Cells(Board):
        row, col = empty_cell
        SetMove(Board, row, col, char)
        value, depth = MiniMaxAlphaBeta(Board, not computer, alpha, beta, depth + 1)
        m = func(m, value)
        Board[row][col] = 0
        if computer:
            alpha = func(alpha, m)
        else:
            beta = func(beta, m)
        if alpha >= beta:
            break
    return m, depth

def Computer_move(Board):
    m = alpha = -INF
    d = beta = INF
    for action in empty_Cells(Board):
        row, col = action
        SetMove(Board, row, col, 1)
        value, depth = MiniMaxAlphaBeta(Board, False, alpha, beta, 0)
        if value > m or (value == m and depth < d):
            result = row, col
            m = value
            d = depth
        Board[row][col] = 0
    return result

def UserInputCoordinates(Board):
    ActionsMap = {
        0: [0, 0], 1: [0, 1], 2: [0, 2],
        3: [1, 0], 4: [1, 1], 5: [1, 2],
        6: [2, 0], 7: [2, 1], 8: [2, 2],
        }
    while True:
        action = input("Enter the Move (Box-0 --> Box-8):\t")
        if action.isdigit() and int(action) in range(size**2):
            action = int(action)
            points = ActionsMap[action]
            return points[0], points[1]
        else:
            print("Wrong Choice...")
    


**Main**

In [ ]:
def main(Board):
    printBoard(Board)
    Finish = False
    while Finish == False:
        print("Your Turn(X): ")
        x,y = UserInputCoordinates(Board)
        SetMove(Board, x, y, HUMAN)
        printBoard(Board)
        if win(Board, HUMAN):
            print("You Win!...")
            Finish = True
        elif GameOver(Board):
            print("Draw...")
            Finish = True
        else:
            print("Computer Turn(O)")
            x, y = Computer_move(Board)
            SetMove(Board, x, y, COMPUTER)
            printBoard(Board)
            if win(Board, COMPUTER):
                print("Computer Win!...")
                Finish = True

if __name__ == "__main__":
    main(Board)
